In [ ]:
from avapi.nuscenes import nuScenesManager


data_dir = "/home/data/nuScenes"
NSM = nuScenesManager(data_dir)
NDM = NSM.get_scene_dataset_by_index(0)

In [ ]:
import json


# load the dataset
json_path = "../scripts/dataset_train.json"
with open(json_path, "r") as file:
    index_dict = json.load(file)
dataset = index_dict["dataset"]


# flatten to 1-d for indexing
dataset_flat = [
    frame
    for scene in dataset.values()
    for agent in scene.values()
    for frame in agent.values()
]

In [ ]:
# investigate the speed in some frames
idx = 2
posit = dataset_flat[idx]["agent_state"]["global"]["position"]
posit_local = dataset_flat[idx]["agent_state"]["local"]["position"]
veloc = dataset_flat[idx]["agent_state"]["global"]["velocity"]
speed = dataset_flat[idx]["agent_state"]["global"]["speed"]
waypoints_3d = dataset_flat[idx]["waypoints_3d"]
waypoints_bev = [(round(pt[0], 1), round(pt[2], 1)) for pt in waypoints_3d.values()]

# print results
print(f"Velo: {veloc}")
print(f"Speed: {speed:.1f}")
print(f"Waypoints BEV: {waypoints_bev}")
print(f"Position: {posit}")
print(f"Position local: {posit_local}")

# test out position in the camera frame
frame = dataset_flat[idx]["frame"]
NDM = NSM.get_scene_dataset_by_name(dataset_flat[idx]["scene"])
cam_calib = NDM.get_calibration(frame=frame, sensor="CAM_FRONT")
ego_state = NDM.get_ego(frame=frame)
posit_camera = ego_state.position.change_reference(cam_calib.reference, inplace=False)
print(f"Position camera: {posit_camera.x}")


# print future position
posit_future = NDM.get_ego(frame=frame + 2).position
posit_future.change_reference(cam_calib.reference, inplace=True)
print(f"Two frame future: {posit_future.x}")

### Investigate position difference vs. velocity CAN reading

In [ ]:
# Investigate position difference vs. velocity CAN reading
vel_by_pos = []
vel_by_can = []
pos = []
dt = 0.5 * 2
for DM in NSM:
    for frame in DM.frames[:-2]:
        ego = DM.get_ego(frame=frame)
        ego_next = DM.get_ego(frame=frame + 2)
        dx = ego_next.position.x - ego.position.x
        pos.append(ego.position.x)
        vel_by_pos.append(dx / dt)
        vel_by_can.append(ego.velocity.x)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


vel_diff = np.array(vel_by_can) - np.array(vel_by_pos)
plt.hist(np.linalg.norm(vel_diff, axis=1), bins=50)
plt.show()


for i in range(3):
    plt.hist(vel_diff[:, i], label=i)
    plt.legend()
plt.show()